In [ ]:
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
import random
import numpy as np
import torch
import sklearn
from sklearn import decomposition
from sklearn.decomposition import PCA
from data_preprocessing import process_data
from dataset import MMCDataset
from model import LightningLSTM

In [ ]:
sorted_data, sorted_length, target_data = process_data()
print(sorted_data.shape)
sorted_data
type(sorted_data)
np.reshape(sorted_data, [-1, 1015])
sorted_data = np.reshape(sorted_data,[-1,1015])
sorted_data.shape

In [ ]:
pca = PCA(n_components=400)
#pca = decomposition.PCA()
data_pca = pca.fit_transform(sorted_data)
data_pca = np.reshape(data_pca, [404, 6, 400])
print(data_pca.shape)

In [ ]:
kf = KFold(n_splits=5)
for index, (train_index, test_index) in enumerate(kf.split(data_pca)):
    random.seed(100)
    np.random.seed(100)
    torch.manual_seed(100)

    X_train = data_pca[train_index]
    X_train_length = sorted_length[train_index]
    y_train = target_data[train_index]

    X_test = data_pca[test_index]
    X_test_length = sorted_length[test_index]
    y_test = target_data[test_index]

    train_dataset = MMCDataset(X_train, X_train_length, y_train)
    test_dataset = MMCDataset(X_test, X_test_length, y_test)
    train_data_loader = DataLoader(train_dataset, batch_size=64)
    test_data_loader = DataLoader(test_dataset, batch_size=64)


    lightning_lstm = LightningLSTM(input_size=train_dataset[0][0].shape[1], hidden_size=32, output_size=train_dataset.y.shape[1])
    pl_trainer = Trainer(max_epochs=100, callbacks=[EarlyStopping(monitor='validation_loss', patience=6)],
                                                    checkpoint_callback=ModelCheckpoint('saved_model/model', monitor='validation_loss',
                                                                                        save_top_k=1, prefix=f'kfold_{index}'))
    pl_trainer.fit(lightning_lstm, train_data_loader, test_data_loader)